#### Data Preparation
Reading the text data, tokenizing and creating sequences of 50 words  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'numpy'

In [2]:
with open('text8', 'r') as file:
    text = file.read()


tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1


input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
X = input_sequences[:,:-1]
y = input_sequences[:,-1]


In [ ]:

y = tf.keras.utils.to_categorical(y, num_classes=total_words)

#### Model Building
Designing an LSTM model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


### Training
Training the above model

In [ ]:
history = model.fit(X, y, epochs=20, verbose=1, validation_split=0.2)

In [ ]:
# Save the model using pickle library
import pickle
model.save('model.h5')
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))


#### Text Generation
Implementing a function to generate text by predicting the next word iteratively.

In [ ]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


print(generate_text("This is an example of", 20, max_sequence_len))
print(generate_text("Deep learning models are", 20, max_sequence_len))
